In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

**<font size="4" color="red">ch3. 연관분석</font>**
- pip install apyori

# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석
- 활용 분야 : 이벤트 미리감지(사기적발,...), 신상품 카테고리 구성

[조건 : left-hand side:오렌지주스]->[결과 : right-hand side:와인]
- 연관분석과 관련된 지표
```
1. 지지도(support) : 얼마나 자주, 함께 나타나는지<br>
    (lhs와 rhs)의 항목수 / 전체 항목 수  = 0.2<br>
2. 신뢰도(confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지<br>
    (lhs->rhs)의 항목 수 / lhs의 항목 수 = 1/2 = 0.5<br>
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인<br>
    (lhs->rhs)의 항목 수 / (lhs의 항목 수*rhs의 항목 수)<br>
    => 0.2 / (0.4*0.6) = 0.2/0.24 = 0.833<br>
    향상도<1 : 기대가 낮다<br>
    향상도>1 : 기대가 높다<br>
```

# 2. 연관분석 구현

In [3]:
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f :
    csvdata = csv.reader(f)
#     print(list(csvdata))
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [4]:
from apyori import apriori
rules = apriori(transaction, # 2차원 데이터
                min_support = 0.15,
                min_confidence=0.1
                )
rules = list(rules)
len(rules)

18

In [6]:
rules[10]

RelationRecord(items=frozenset({'소주', '콜라'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주', '콜라'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [11]:
for rule in rules :
    support = rule[1]
    order_st = rule[2]
    for item in order_st : 
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            print("{} => {}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                                   round(confidence, 2),
                                                   round(lift,2)))

맥주 => 콜라	 0.4	 1.0	 1.25
콜라 => 맥주	 0.4	 0.5	 1.25
소주 => 콜라	 0.6	 1.0	 1.25
콜라 => 소주	 0.6	 0.75	 1.25
콜라 => 소주, 맥주	 0.2	 0.25	 1.25
소주, 맥주 => 콜라	 0.2	 1.0	 1.25
맥주 => 콜라, 와인	 0.2	 0.5	 1.25
콜라 => 맥주, 와인	 0.2	 0.25	 1.25
맥주, 와인 => 콜라	 0.2	 1.0	 1.25
콜라, 와인 => 맥주	 0.2	 0.5	 1.25
소주 => 오렌지주스, 콜라	 0.2	 0.33	 1.67
콜라 => 소주, 오렌지주스	 0.2	 0.25	 1.25
소주, 오렌지주스 => 콜라	 0.2	 1.0	 1.25
오렌지주스, 콜라 => 소주	 0.2	 1.0	 1.67
콜라 => 소주, 와인	 0.2	 0.25	 1.25
소주, 와인 => 콜라	 0.2	 1.0	 1.25


In [14]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs', 'rhs', '지지도', '신뢰도', '향상도'])
# rules_df.loc[0] = ['와인', '오렌지', 0.15, 0.5, 1.1] 식으로 for문내에서 데이터 추가
idx = 0
for rule in rules :
    support = rule[1]
    order_st = rule[2]
    for item in order_st : 
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
            idx+=1
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False)            

,lhs,rhs,지지도,신뢰도,향상도
13,"오렌지주스, 콜라",소주,0.2,1.00,1.67
10,소주,"오렌지주스, 콜라",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"소주, 맥주",콜라,0.2,1.00,1.25
8,"맥주, 와인",콜라,0.2,1.00,1.25
12,"소주, 오렌지주스",콜라,0.2,1.00,1.25
15,"소주, 와인",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 경주/전주 여행 자료 연관분석

In [28]:
import pandas as pd
from konlpy.tag import Hannanum, Kkma, Komoran
df = pd.read_csv('data/naver_kin.csv', sep='\t')
total_text_list = df['total_text'].to_list()
# total_text_list[:2]
analyzer = Komoran()
total_noun_list = []
# select_pos = ['NC', 'NQ'] # Hannanum 보통명사, 고유명사
select_pos = ['NNP','NNG'] # Kkma, Komoran 보통명사, 고유명사
불용어 = {'여행'}
for total_text in total_text_list :
#     total_noun = analyzer.nouns(total_text)
    total_noun = [token for token, tag in analyzer.pos(total_text)
                     if tag in select_pos and
                         token not in 불용어 and
                         len(token)>1]
    total_noun_list.append(total_noun)
total_noun_list[:2]

[['전주',
  '한곳',
  '추천',
  '추억',
  '체험',
  '감성',
  '추억',
  '테마',
  '박물관',
  '시간',
  '전북',
  '전북',
  '투어',
  '패스',
  '통합',
  '이용',
  '여행지',
  '체험',
  '카페',
  '이용',
  '추가',
  '전주',
  '필수',
  '편안',
  '감사'],
 ['전주',
  '전주',
  '사람과',
  '호텔',
  '가격',
  '얼마',
  '정도',
  '맛집',
  '카페',
  '추천',
  '안녕하세요',
  '전주',
  '계획',
  '한옥마을',
  '근처']]

In [31]:
%%time
rules = apriori(total_noun_list, min_support=0.15, min_confidence=0.3)
rules = list(rules)
len(rules)

CPU times: total: 125 ms
Wall time: 106 ms


193

In [38]:
for rule in rules :
    support = rule[1]
    order_st = rule[2]
    for item in order_st : 
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
            idx+=1
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False, inplace=True)
rules_df = rules_df.reset_index(drop=True)

In [36]:
pd.options.display.max_rows

60

In [39]:
rules_df.head(60)

,lhs,rhs,지지도,신뢰도,향상도
0,제주도,"맛집, 여수",0.152,1.0,5.99
1,제주도,"여수, 주가",0.152,1.0,5.99
2,제주도,"여수, 코스",0.152,1.0,5.99
3,제주도,"여수, 한곳",0.152,1.0,5.99
4,"국내, 주가","맛집, 여수",0.157,1.0,5.99
5,"국내, 한곳","맛집, 여수",0.157,1.0,5.99
6,"국내, 주가","여수, 코스",0.157,1.0,5.99
7,"국내, 주가","여수, 한곳",0.157,1.0,5.99
8,"국내, 한곳","여수, 주가",0.157,1.0,5.99
9,"국내, 한곳","여수, 코스",0.157,1.0,5.99
